In [ ]:
!pip uninstall -y catboost numpy
!pip install --no-cache-dir numpy catboost

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 34.7 MB/s eta 0:00:00


In [ ]:
!pip uninstall numpy -y
!pip install numpy

Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Successfully uninstalled numpy-2.2.4
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.2.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.


In [ ]:
pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving Judgment_Embeddings_InLegalBERT.xlsx to Judgment_Embeddings_InLegalBERT.xlsx


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# load dataset
def load_dataset(file_path):
    return pd.read_excel(file_path)

#Preprocessing data into X(features), y(target)
def preprocess_data(df):
    X = df.drop(columns=["Label"]).values
    y = df["Label"].values
    return X, y

# split dataset
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

#hyperparameter tuning using RandomizedSearchCV
def tune_hyperparameters(model, param_grid, X_train, y_train, n_iter=20, cv=5):
    if param_grid:
        search = RandomizedSearchCV(model, param_distributions=param_grid,
                                    n_iter=n_iter, cv=cv, n_jobs=-1, verbose=1)
        search.fit(X_train, y_train)
        return search.best_estimator_
    return model

#train and evaluate a model
def train_and_evaluate(model, model_name, param_grid, X_train, X_test, y_train, y_test):
    model = tune_hyperparameters(model, param_grid, X_train, y_train)
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(f"{model_name} Results:")
    print(f"Train Accuracy: {train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print("Classification Report:(Train)\n", classification_report(y_train, y_train_pred))
    print("Classification Report:(Test)\n", classification_report(y_test, y_test_pred))

    return model, train_acc, test_acc

def svm_classifier(X_train, X_test, y_train, y_test):
    model = SVC()
    param_grid = {"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]}
    return train_and_evaluate(model, "SVM", param_grid, X_train, X_test, y_train, y_test)

def decision_tree_classifier(X_train, X_test, y_train, y_test):
    model = DecisionTreeClassifier()
    param_grid = {"max_depth": [5, 10, 20, None]}
    return train_and_evaluate(model, "Decision Tree", param_grid, X_train, X_test, y_train, y_test)

def random_forest_classifier(X_train, X_test, y_train, y_test):
    model = RandomForestClassifier()
    param_grid = {"n_estimators": [50, 100, 200], "max_depth": [5, 10, None]}
    return train_and_evaluate(model, "Random Forest", param_grid, X_train, X_test, y_train, y_test)

def xgboost_classifier(X_train, X_test, y_train, y_test):
    model = XGBClassifier()
    param_grid = {"n_estimators": [50, 100], "learning_rate": [0.01, 0.1, 0.2]}
    return train_and_evaluate(model, "XGBoost", param_grid, X_train, X_test, y_train, y_test)

def adaboost_classifier(X_train, X_test, y_train, y_test):
    model = AdaBoostClassifier()
    param_grid = {"n_estimators": [50, 100]}
    return train_and_evaluate(model, "AdaBoost", param_grid, X_train, X_test, y_train, y_test)

file_path = "Judgment_Embeddings_InLegalBERT.xlsx"
df = load_dataset(file_path)
X, y = preprocess_data(df)
X_train, X_test, y_train, y_test = split_data(X, y)

svm_classifier(X_train, X_test, y_train, y_test)
decision_tree_classifier(X_train, X_test, y_train, y_test)
random_forest_classifier(X_train, X_test, y_train, y_test)
xgboost_classifier(X_train, X_test, y_train, y_test)
adaboost_classifier(X_train, X_test, y_train, y_test)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


SVM Results:
Train Accuracy: 0.7755
Test Accuracy: 0.6718
Classification Report:(Train)
               precision    recall  f1-score   support

           0       0.72      0.92      0.81      1088
           1       0.83      0.86      0.84       495
           2       0.83      0.82      0.83       297
           3       0.00      0.00      0.00         7
           4       0.87      0.37      0.52       108
           5       0.00      0.00      0.00        84
           6       0.86      0.95      0.90       207
           7       1.00      0.50      0.67         6
           8       0.91      0.36      0.51        56
           9       0.89      0.27      0.42       175
          10       0.00      0.00      0.00         6
          11       0.95      0.87      0.91        23
          12       0.00      0.00      0.00        20
          13       0.00      0.00      0.00        11

    accuracy                           0.78      2583
   macro avg       0.56      0.42      0.46  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Decision Tree Results:
Train Accuracy: 0.5919
Test Accuracy: 0.5542
Classification Report:(Train)
               precision    recall  f1-score   support

           0       0.55      0.93      0.69      1088
           1       0.63      0.65      0.64       495
           2       0.73      0.30      0.42       297
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00       108
           5       0.00      0.00      0.00        84
           6       1.00      0.52      0.69       207
           7       0.00      0.00      0.00         6
           8       1.00      0.02      0.04        56
           9       0.67      0.01      0.02       175
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00        23
          12       0.00      0.00      0.00        20
          13       0.00      0.00      0.00        11

    accuracy                           0.59      2583
   macro avg       0.33      0.17  

...

XGBoost Results:
Train Accuracy: 0.9988
Test Accuracy: 0.6517
Classification Report:(Train)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1088
           1       1.00      1.00      1.00       495
           2       1.00      1.00      1.00       297
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00       108
           5       1.00      1.00      1.00        84
           6       1.00      1.00      1.00       207
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00        56
           9       1.00      1.00      1.00       175
          10       1.00      1.00      1.00         6
          11       1.00      1.00      1.00        23
          12       1.00      1.00      1.00        20
          13       1.00      1.00      1.00        11

    accuracy                           1.00      2583
   macro avg       1.00      1.00      1.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

AdaBoost Results:
Train Accuracy: 0.4150
Test Accuracy: 0.4071
Classification Report:(Train)
               precision    recall  f1-score   support

           0       0.49      0.49      0.49      1088
           1       0.34      0.52      0.41       495
           2       0.29      0.47      0.36       297
           3       0.00      0.00      0.00         7
           4       0.28      0.05      0.08       108
           5       0.00      0.00      0.00        84
           6       0.88      0.60      0.72       207
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00        56
           9       0.13      0.06      0.08       175
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00        23
          12       0.00      0.00      0.00        20
          13       0.00      0.00      0.00        11

    accuracy                           0.42      2583
   macro avg       0.17      0.16      0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

(AdaBoostClassifier(), 0.41502129307007357, 0.4071207430340557)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# load dataset
def load_dataset(file_path):
    return pd.read_excel(file_path)

#Preprocessing data into X(features), y(target)
def preprocess_data(df):
    X = df.drop(columns=["Label"]).values
    y = df["Label"].values
    return X, y

# split dataset
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

#hyperparameter tuning using RandomizedSearchCV
def tune_hyperparameters(model, param_grid, X_train, y_train, n_iter=20, cv=5):
    if param_grid:
        search = RandomizedSearchCV(model, param_distributions=param_grid,
                                    n_iter=n_iter, cv=cv, n_jobs=-1, verbose=1)
        search.fit(X_train, y_train)
        return search.best_estimator_
    return model

#train and evaluate a model
def train_and_evaluate(model, model_name, param_grid, X_train, X_test, y_train, y_test):
    model = tune_hyperparameters(model, param_grid, X_train, y_train)
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(f"{model_name} Results:")
    print(f"Train Accuracy: {train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print("Classification Report:(Train)\n", classification_report(y_train, y_train_pred))
    print("Classification Report:(Test)\n", classification_report(y_test, y_test_pred))

    return model, train_acc, test_acc

def naive_bayes_classifier(X_train, X_test, y_train, y_test):
    model = GaussianNB()
    return train_and_evaluate(model, "Naïve Bayes", None, X_train, X_test, y_train, y_test)

file_path = "Judgment_Embeddings_InLegalBERT.xlsx"
df = load_dataset(file_path)
X, y = preprocess_data(df)
X_train, X_test, y_train, y_test = split_data(X, y)

naive_bayes_classifier(X_train, X_test, y_train, y_test)

Naïve Bayes Results:
Train Accuracy: 0.5029
Test Accuracy: 0.4520
Classification Report:(Train)
               precision    recall  f1-score   support

           0       0.82      0.30      0.44      1088
           1       0.81      0.54      0.64       495
           2       0.49      0.75      0.59       297
           3       1.00      1.00      1.00         7
           4       0.23      0.67      0.34       108
           5       0.19      0.37      0.25        84
           6       0.86      0.86      0.86       207
           7       1.00      1.00      1.00         6
           8       0.25      0.79      0.38        56
           9       0.28      0.55      0.37       175
          10       1.00      1.00      1.00         6
          11       0.44      0.96      0.60        23
          12       0.12      0.65      0.20        20
          13       0.83      0.91      0.87        11

    accuracy                           0.50      2583
   macro avg       0.59      0.74    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(GaussianNB(), 0.502903600464576, 0.4520123839009288)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# load dataset
def load_dataset(file_path):
    return pd.read_excel(file_path)

#Preprocessing data into X(features), y(target)
def preprocess_data(df):
    X = df.drop(columns=["Label"]).values
    y = df["Label"].values
    return X, y

# split dataset
def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

#hyperparameter tuning using RandomizedSearchCV
def tune_hyperparameters(model, param_grid, X_train, y_train, n_iter=20, cv=5):
    if param_grid:
        search = RandomizedSearchCV(model, param_distributions=param_grid,
                                    n_iter=n_iter, cv=cv, n_jobs=-1, verbose=1)
        search.fit(X_train, y_train)
        return search.best_estimator_
    return model

#train and evaluate a model
def train_and_evaluate(model, model_name, param_grid, X_train, X_test, y_train, y_test):
    model = tune_hyperparameters(model, param_grid, X_train, y_train)
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(f"{model_name} Results:")
    print(f"Train Accuracy: {train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print("Classification Report:(Train)\n", classification_report(y_train, y_train_pred))
    print("Classification Report:(Test)\n", classification_report(y_test, y_test_pred))

    return model, train_acc, test_acc

def mlp_classifier(X_train, X_test, y_train, y_test):
    model = MLPClassifier(max_iter=500)
    param_grid = {"hidden_layer_sizes": [(50,), (100,), (50, 50)], "activation": ["relu", "tanh"]}
    return train_and_evaluate(model, "MLP", param_grid, X_train, X_test, y_train, y_test)

file_path = "Judgment_Embeddings_InLegalBERT.xlsx"
df = load_dataset(file_path)
X, y = preprocess_data(df)
X_train, X_test, y_train, y_test = split_data(X, y)

mlp_classifier(X_train, X_test, y_train, y_test)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Results:
Train Accuracy: 0.9988
Test Accuracy: 0.6130
Classification Report:(Train)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1088
           1       1.00      1.00      1.00       495
           2       1.00      1.00      1.00       297
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00       108
           5       1.00      1.00      1.00        84
           6       1.00      1.00      1.00       207
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00        56
           9       1.00      1.00      1.00       175
          10       1.00      1.00      1.00         6
          11       1.00      1.00      1.00        23
          12       1.00      1.00      1.00        20
          13       1.00      1.00      1.00        11

    accuracy                           1.00      2583
   macro avg       1.00      1.00      1.00  

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and bei

(MLPClassifier(activation='tanh', hidden_layer_sizes=(50,), max_iter=500),
 0.9988385598141696,
 0.6130030959752322)